# TensorRec recommender engine:
## Prototyping of a recommender system in Python using TensorRec including input data manipulation, algorithm design, and usage for prediction.

#### TensorRec is a Python package for building recommender systems. A TensorRec recommender system consumes three pieces of input data: user features, item features, and interactions. Based on the user/item features, the system will predict which items to recommend. The interactions are used when fitting the model: predictions are compared to the interactions and a loss/penalty is calculated, which the system learns to decrease. As we prototype our system, we tackle three major situations: how we handle interactions, how we handle features, and how we structure the recommender itself.


<img src="https://miro.medium.com/max/1400/1*YotDpHjvGL8xK91ZggthbA.png" />


https://towardsdatascience.com/getting-started-with-recommender-systems-and-tensorrec-8f50a9943eef

### Raw ratings load :Each row represents a single rating: one user and one item. We’ll be using these ratings(frequency of purchase of each item) as our interactions between the user and the product.

In [1]:
from google.cloud import bigquery
import time
t0 = time.time()
limite = 189857*3 # 18,985,770  # corre en mi local %0.1 del total de la muestra

# functions
def get_data_BQ(sql):
    client = bigquery.Client()
    df = client.query(sql).to_dataframe()
    return(df)

In [2]:
sql =  '''SELECT USERID as ID_CTE, ID_FAM as ID_CLAS1, FREQUENCY as FREQUENCY
FROM `rmf2gcp.RawData.Workflow_aggregado`
WHERE id_table_dem <=  ''' + str(limite) + ''' ORDER BY USERID'''

print(sql)
raw_ratings = get_data_BQ(sql)
raw_ratings = raw_ratings.values.tolist() # pues vamos a seguir su mala practica de hacer una lista de listas 
raw_ratings[0:7]

SELECT USERID as ID_CTE, ID_FAM as ID_CLAS1, FREQUENCY as FREQUENCY
FROM `rmf2gcp.RawData.Workflow_aggregado`
WHERE id_table_dem <=  569571 ORDER BY USERID


[[10002, 665040, 1],
 [10002, 248001, 1],
 [10002, 219060, 1],
 [10002, 232021, 1],
 [10002, 222011, 1],
 [10002, 124002, 1],
 [10002, 229008, 1]]

### Iterate through the input to map Item and User IDs to new internal IDs
### The new internal IDs will be created by the defaultdict on insertion

In [3]:
import collections
idcte_to_internal_user_ids = collections.defaultdict(lambda: len(idcte_to_internal_user_ids))
idfam1_to_internal_item_ids = collections.defaultdict(lambda: len(idfam1_to_internal_item_ids))
for row in raw_ratings:
    row[0] = idcte_to_internal_user_ids[int(row[0])]
    row[1] = idfam1_to_internal_item_ids[int(row[1])]
    row[2] = float(row[2])    # esta operacion esta de más 
n_users = len(idcte_to_internal_user_ids)
n_items = len(idfam1_to_internal_item_ids)
print(n_users)
print(n_items)

569571
2307


In [4]:
print( [ (key, value) for key, value  in idfam1_to_internal_item_ids.items() ][0:7])
print( [ (key, value) for key, value  in idcte_to_internal_user_ids.items() ][0:7])

[(665040, 0), (248001, 1), (219060, 2), (232021, 3), (222011, 4), (124002, 5), (229008, 6)]
[(10002, 0), (10006, 1), (10011, 2), (10036, 3), (10039, 4), (10043, 5), (10047, 6)]


In [5]:
from collections import defaultdict
import csv
import numpy 
import random
from scipy import sparse
from sklearn.preprocessing import MultiLabelBinarizer

### At this point, we’ll break the ratings in to a training and test set by shuffling and splitting the ratings. Our prototypes will be trained on the training set, and we’ll evaluate their success using the test set. Splitting the train/test sets at random like this is crude, and there are more rigorous techniques for model evaluation, but it is quick and clear for the purposes of this example. 

In [6]:
# Shuffle the ratings and split them in to train/test sets 80%/20%
random.shuffle(raw_ratings)  # Shuffles the list in-place
cutoff = int(.8 * len(raw_ratings))
train_ratings = raw_ratings[:cutoff]
test_ratings = raw_ratings[cutoff:]


### Next, we reorganize these ratings in to a Scipy sparse matrix. In this matrix, every row represents a user and every column is an item. The [i, j]th value in this matrix is User i’s interaction with Item j.

In [7]:

# This method converts a list of (user, item, rating) to a sparse matrix
def interactions_list_to_sparse_matrix(interactions):
    users_column, items_column, ratings_column, = zip(*interactions)
    return sparse.coo_matrix((ratings_column, (users_column, items_column)),
                             shape=(n_users, n_items))


# Create sparse matrices of interaction data
sparse_train_ratings = interactions_list_to_sparse_matrix(raw_ratings)
sparse_test_ratings = interactions_list_to_sparse_matrix(test_ratings)



In [8]:
sparse_train_ratings

<569571x2307 sparse matrix of type '<class 'numpy.float64'>'
	with 11021877 stored elements in COOrdinate format>

### TensorRec library runs on TensorFlow so we install a compatible version of TensorFlow 
### Both TensorFlow and TensorRec can be installed using !pip

In [9]:
#!pip install "tensorflow==1.13.1"
print(n_users)

569571


In [10]:
import tensorflow as tf
print(tf.__version__)

1.15.3-dlenv_tfe


In [11]:
#!pip install tensorrec --ignore-installed

In [12]:
import tensorrec

## Collaborative Filter Prototype
### A collaborative filter is an algorithm that learns which users have similar tastes and recommends items to a user based on what similar users have liked. A common way to do this is through matrix factorization. In matrix factorization, we have to learn two matrices (user representations and item representations) that, when multiplied together, approximate the interactions:
#### TensorRec will perform matrix factorization by default if it is given only identity matrices as user/item features. These identity matrices are often called “indicator features.”

In [13]:
# Construct indicator features for users and items
user_indicator_features = sparse.identity(n_users)
item_indicator_features = sparse.identity(n_items)

# Build a matrix factorization collaborative filter model
cf_model = tensorrec.TensorRec(n_components=5)

# Fit the collaborative filter model
print("Training collaborative filter")
cf_model.fit(interactions=sparse_train_ratings,
             user_features=user_indicator_features,
             item_features=item_indicator_features)

Training collaborative filter


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## Loss Graphs
### One way we can configure our TensorRec system is by changing the loss graph. The loss graph takes in predictions and interactions and calculates a penalty (loss) that the system will try to decrease as it learns.
#### WMRB, which stands for “weighted margin-rank batch,” works by taking a random sample of items the user hasn’t interacted with and comparing their predictions to items the user likes. Over time, this pushes items a user likes to the top of the rankings. We can try using different loss graphs like WARP

# Adding Metadata Features
## To continue experimenting, we should try to make use of other data available to us. We will try using User Demographic data

In [14]:
# To improve the recommendations, lets read in the user demographic data
sql = """
SELECT USERID, FEATURES, STATE
FROM `rmf2gcp.RawData.demographics_features` 
WHERE id_table_dem <= """ + str(limite)
print(sql)


SELECT USERID, FEATURES, STATE
FROM `rmf2gcp.RawData.demographics_features` 
WHERE id_table_dem <= 569571


In [15]:
raw_user_metadata = get_data_BQ(sql)
print(raw_user_metadata.head(800))
print(raw_user_metadata.dtypes)
print(raw_user_metadata.columns)
raw_user_metadata_header = ['USERID', 'FEATURES', 'STATE']
raw_user_metadata_header

     USERID FEATURES                      STATE
0     10006   81,C,M  SINALOA                  
1     10050   75,C,M  SINALOA                  
2     10059   87,C,M  SINALOA                  
3     10066   77,C,F  SINALOA                  
4     10097   75,C,M  SINALOA                  
..      ...      ...                        ...
795   20397   63,S,M  SINALOA                  
796   20405   80,C,F  SINALOA                  
797   20407   84,S,F  SINALOA                  
798   20438   74,C,M  SINALOA                  
799   20440   75,C,M  SINALOA                  

[800 rows x 3 columns]
USERID       int64
FEATURES    object
STATE       object
dtype: object
Index(['USERID', 'FEATURES', 'STATE'], dtype='object')


['USERID', 'FEATURES', 'STATE']

In [16]:
raw_user_metadata = raw_user_metadata.values.tolist()
raw_user_metadata[0:7]

[[10006, '81,C,M', 'SINALOA                  '],
 [10050, '75,C,M', 'SINALOA                  '],
 [10059, '87,C,M', 'SINALOA                  '],
 [10066, '77,C,F', 'SINALOA                  '],
 [10097, '75,C,M', 'SINALOA                  '],
 [10104, '78,C,F', 'SINALOA                  '],
 [10110, '88,V,M', 'SINALOA                  ']]

In [17]:
17

17

In [18]:
18

18

In [19]:
19

19

In [20]:
20

20

In [21]:
21

21

In [22]:
22

22

In [23]:
23

23

In [24]:
24

24

### First, we’ll want to read this data, map the movies to our internal IDs, and keep track of the features for each user. Then we’ll binarize the feature  labels using Scikit’s MultiLabelBinarizer. The binarized output will be our features for our new recommender system.

In [25]:
# Map the features IDs to our internal IDs and keep track of the gender and age
temp_string = ''
temp_list = []
count = 1
user_id_by_internal_id = {}
user_features_by_internal_id = {}
for row in raw_user_metadata:
    temp_string = ''
    temp_list = []
    

    temp_string = str(row[0])
    temp_list = row[1].split(',')
    #print(count)
    #print(temp_string)
    #print(temp_list)
    row[0] = idfam1_to_internal_item_ids[int(temp_string)]  # Map to IDs
    row[1] = temp_list  # Split up
    user_id_by_internal_id[temp_string] = temp_string
    user_features_by_internal_id[int(temp_string)] = row[1]
    count+=1
# Look at an example user metadata row
print("Raw metadata example:\n{}\n{}".format(raw_user_metadata_header, 
                                             raw_user_metadata[0]))



Raw metadata example:
['USERID', 'FEATURES', 'STATE']
[2307, ['81', 'C', 'M'], 'SINALOA                  ']


In [26]:
[ (key, value) for key,value in user_features_by_internal_id.items() ][0:7]

[(10006, ['81', 'C', 'M']),
 (10050, ['75', 'C', 'M']),
 (10059, ['87', 'C', 'M']),
 (10066, ['77', 'C', 'F']),
 (10097, ['75', 'C', 'M']),
 (10104, ['78', 'C', 'F']),
 (10110, ['88', 'V', 'M'])]

###  Build a list of features where the index is the internal user ID and the value is a list of features

In [27]:
user_feat = [user_features_by_internal_id[internal_id]
                for internal_id in user_features_by_internal_id]

In [28]:
user_feat[0:7]

[['81', 'C', 'M'],
 ['75', 'C', 'M'],
 ['87', 'C', 'M'],
 ['77', 'C', 'F'],
 ['75', 'C', 'M'],
 ['78', 'C', 'F'],
 ['88', 'V', 'M']]

In [29]:
29 #perdido

29

In [30]:
# Transform the features into binarized labels using scikit's MultiLabelBinarizer
user_features = MultiLabelBinarizer().fit_transform(user_feat)
n_features = user_features.shape[1]

In [31]:
user_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
32 #perdido

32

### Coerce the user features to a sparse matrix, which TensorRec expects


In [33]:
user_features_mat = sparse.coo_matrix(user_features)
user_features_mat

<569571x96 sparse matrix of type '<class 'numpy.int64'>'
	with 1708713 stored elements in COOrdinate format>

### It’s not as good as the ranking collaborative filter but it’s significantly more effective if we add it to the base collaborative filter.
#### There is a major weakness to this system: user feature alone are not very descriptive and are not enough information to make an informed recommendation. If we had more descriptive metadata and item metadata (views, clicks, basket information etc.) we may have more success with this content-based recommender system.
#### On the other hand, there is a major strength to this system: by relying on only metadata features, and not using indicator features, we can recommend products which were not present when training the model. Similarly, if we have valuable user metadata we can avoid using user indicator features and make predictions for users who’ve never interacted with a product before. This is called “cold-start” recommendation.

# Hybrid recommender: 
## Hybrid recommender systems combine two or more recommendation strategies in different ways to benefit from their complementary advantages.
### Let’s combine these two: we’ll use indicator features to get the strengths of a collaborative filter, and we’ll also use the content features to take advantage of the metadata. This combination of collaborative filtering and content-based recommendation is the hybrid model.


#### We do this by stacking the two sets of features together:

In [34]:
# Try concatenating the user features on to the indicator features for a hybrid recommender system
full_user_features = sparse.hstack([user_indicator_features, user_features_mat])
full_user_features

<569571x569667 sparse matrix of type '<class 'numpy.float64'>'
	with 2278284 stored elements in COOrdinate format>

In [35]:
print("Training hybrid recommender")
hybrid_model = tensorrec.TensorRec(
    n_components=5
)
hybrid_model.fit(interactions=sparse_train_ratings,
                 user_features=full_user_features,
                 item_features=item_indicator_features,
                 n_sampled_items=int(n_items * .01))


Training hybrid recommender


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### This performs the best even though we are using trivial features from users. If we have more metadata, we can expect larger impact of the Hybrid recommender

In [36]:
print("Hybrid recommender:")
predicted_ranks = hybrid_model.predict_rank(user_features=full_user_features,
                                            item_features=item_indicator_features)
def check_results(ranks):
    train_recall_at_10 = tensorrec.eval.recall_at_k(
        test_interactions=sparse_train_ratings,
        predicted_ranks=ranks,
        k=10
    ).mean()
    test_recall_at_10 = tensorrec.eval.recall_at_k(
        test_interactions=sparse_test_ratings,
        predicted_ranks=ranks,
        k=10
    ).mean()
    print("Performance metrics: Train: {:.4f} Test: {:.4f}".format(train_recall_at_10,
                                                            test_recall_at_10))

check_results(predicted_ranks)

Hybrid recommender:
Performance metrics: Train: 0.0218 Test: 0.0238


### Performance of the model: first prototype
#### To do this, we’ll look at a metric called “recall at K.” Recall@K says, for the average user, what percentage of their test items made it in to the top K in the predicted rankings.
#### Recall@K is a nice metric for many recommender systems because it emulates the behavior of a recommendation product. Before calculating the recall, we’ll want to decide which interactions should count as a “purchased item.” In this case, choose to use all ratings of at least 1.0 as “liked products” and ignore the rest. 

In [37]:
# Create sets of train/test interactions that are only frequency > 1 since these represent the products that have been purchased 
sparse_train_ratings_1plus = sparse_train_ratings.multiply(sparse_train_ratings >= 1)
sparse_test_ratings_1plus = sparse_test_ratings.multiply(sparse_test_ratings >= 1)


# This method consumes item ranks for each user and prints out train/test metrics
def check_results(ranks):
    train_recall_at_10 = tensorrec.eval.recall_at_k(
        test_interactions=sparse_train_ratings_1plus,
        predicted_ranks=ranks,
        k=10
    ).mean()
    test_recall_at_10 = tensorrec.eval.recall_at_k(
        test_interactions=sparse_test_ratings_1plus,
        predicted_ranks=ranks,
        k=10
    ).mean()
    print("Performance metrics: Train: {:.4f} Test: {:.4f}".format(train_recall_at_10,
                                                            test_recall_at_10))


# Check the results of the MF CF model
print("Matrix factorization collaborative filter:")
predicted_ranks = cf_model.predict_rank(user_features=user_indicator_features,
                                        item_features=item_indicator_features)
check_results(predicted_ranks)

Matrix factorization collaborative filter:
Performance metrics: Train: 0.0044 Test: 0.0044


In [38]:
38 #perdido

38

In [39]:
39 #perdido

39

In [40]:
40 #perdido

40

In [41]:
41 #perdido

41

In [42]:
42 #perdido

42

In [43]:
43 #perdido

43

## Content-based Recommendation
### Now that we have metadata about our user, one thing we can try is to recommend based solely on the user metadata.
### To do this, we will configure a TensorRec model to use a pass-through representation graph for item features. For us, this means that the user representations will be the same as the user features that are passed in (just the user information like gender, age etc.) and the item representations will reflect how much the item suits that particular set of user features.
#### Ideal case is when we would have item metadata as well: because that would have a greater impact on making the recommendation better- also help solving the cold start problem. There is a major weakness to this system: these features alone are not very descriptive and are not enough information to make an informed recommendation.


In [44]:
# Fit a content-based model using the user features
print("Training content-based recommender")
content_model = tensorrec.TensorRec(
    n_components=n_features,
   user_repr_graph=tensorrec.representation_graphs.FeaturePassThroughRepresentationGraph()
    
)


Training content-based recommender


In [45]:
content_model.fit(interactions=sparse_train_ratings_1plus,
                  user_features=user_features_mat,
                  item_features=item_indicator_features,
                  n_sampled_items=int(n_items * .01))

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [46]:
# Check the results of the content-based model
print("Content-based recommender:")
predicted_ranks = content_model.predict_rank(user_features=user_features_mat,
                                             item_features=item_indicator_features)
check_results(predicted_ranks)

Content-based recommender:
Performance metrics: Train: 0.0252 Test: 0.0271


In [47]:
47 #perdido

47

In [48]:
48 # perdido 

48

In [49]:
49 # perdido

49

### It’s not as good as the ranking collaborative filter but it’s significantly more effective if we add it to the base collaborative filter.
#### There is a major weakness to this system: user feature alone are not very descriptive and are not enough information to make an informed recommendation. If we had more descriptive metadata and item metadata (views, clicks, basket information etc.) we may have more success with this content-based recommender system.
#### On the other hand, there is a major strength to this system: by relying on only metadata features, and not using indicator features, we can recommend products which were not present when training the model. Similarly, if we have valuable user metadata we can avoid using user indicator features and make predictions for users who’ve never interacted with a product before. This is called “cold-start” recommendation.

## Making recommendations
### We do this by passing the user’s feature vector and all the item features to predict_rank() and examining the resulting rankings

In [50]:
# Pull user features out of the user features matrix and predict for just that user
id_coppel =  12198# el cliente con mas compras en la muestra del %.01 # 18988462 # el cliente con mas compras coppel
id_internal = idcte_to_internal_user_ids[id_coppel]
print('id_internal {} , para usuario {}'.format(id_internal, id_coppel))
u_features = sparse.csr_matrix(user_indicator_features)[id_internal  ] # el menos uno es importante 
u_rankings = hybrid_model.predict_rank(user_features=u_features,
                                          item_features=item_indicator_features)[0]

# Get internal IDs of User 432's top 10 recommendations
# These are sorted by item ID, not by rank
# This may contain items with which User 432 has already interacted
k = 10
u_top_ten_recs = numpy.where(u_rankings <= k)[0]
u_top_ten_recs_foo = u_rankings[0:k]# yo creo que los rankings son mas bien
print("User x: Item recommendations:")
u_top_ten_recs = [ key for key,value in idfam1_to_internal_item_ids.items() if value in u_top_ten_recs ]
u_top_ten_recs.sort()
u_top_ten_recs_foo = [ key for key, value in idfam1_to_internal_item_ids.items() if value in u_top_ten_recs_foo]
u_top_ten_recs_foo.sort()
print(u_top_ten_recs)
print(u_top_ten_recs_foo)

id_internal 375 , para usuario 12198
User x: Item recommendations:
[222019, 330011, 387302, 399001, 440006, 594061, 594184, 748103, 802020, 885199]
[106233, 146004, 198006, 320112, 448048, 625020, 647093, 845022, 865028, 950003]


#### The value of the range over which the recommender should iterate has to be the same as the # of the users

In [51]:
# Pull user features out of the user features matrix and predict for just all users
for user in range(0, n_users, 10000):
    u_features = sparse.csr_matrix(full_user_features)[user]
    u_rankings = hybrid_model.predict_rank(user_features=u_features,
                                          item_features=item_indicator_features)[0]
    u_top_ten_recs = numpy.where(u_rankings <= 10)[0]
    print("User"+str(user)+": Item recommendations:")
    print(u_top_ten_recs)


User0: Item recommendations:
[ 386  505  525  844 1099 1148 1210 1823 1912 2158]
User10000: Item recommendations:
[  40  270  310  336  507  771 1016 1097 1440 2188]
User20000: Item recommendations:
[ 218  239  517 1121 1262 1611 1917 1961 2057 2284]
User30000: Item recommendations:
[  87  567  624  894 1039 1129 1227 1230 1914 2027]
User40000: Item recommendations:
[ 809  936  980 1052 1146 1282 1481 1681 2032 2169]
User50000: Item recommendations:
[  54  164  215  312  408 1023 1446 1607 1709 2150]
User60000: Item recommendations:
[  55  452 1027 1099 1105 1318 1835 1842 1955 2046]
User70000: Item recommendations:
[ 295  549  628  824 1035 1379 1421 1556 1662 2298]
User80000: Item recommendations:
[  52  200  325  351  483  572  876  885 1647 1897]
User90000: Item recommendations:
[  87  441  894  973 1129 1177 1946 2027 2181 2229]
User100000: Item recommendations:
[ 499  646  684 1281 1312 1326 1871 1947 2019 2020]
User110000: Item recommendations:
[ 179  210  682  732  811 1032 110

#### Converting Internal IDs back to the original IDs

In [52]:
for user in range(5):
    u_features = sparse.csr_matrix(full_user_features)[user]
    u_rankings = hybrid_model.predict_rank(user_features=u_features,
                                          item_features=item_indicator_features)[0]
    u_top_ten_recs = numpy.where(u_rankings <= 10)[0]
    print("User "+str(list(idcte_to_internal_user_ids.keys())[list(idcte_to_internal_user_ids.values()).index(user)])+": Item recommendations:")
    #print(list(idcte_to_internal_user_ids.keys())[list(idcte_to_internal_user_ids.values()).index(user)]) 
    for m in u_top_ten_recs:
        print(list(idfam1_to_internal_item_ids.keys())[list(idfam1_to_internal_item_ids.values()).index(m)]) 



User 10002: Item recommendations:
301020
774255
775257
326113
844020
245010
701065
381004
305145
210003
User 10006: Item recommendations:
874198
202014
210029
316019
596061
295030
310121
965002
582010
501009
User 10011: Item recommendations:
418073
491005
646266
380535
456040
990001
739104
194085
501018
266004
User 10036: Item recommendations:
423073
301002
295028
307479
308012
807027
405004
210090
399099
415015
User 10039: Item recommendations:
541079
515025
403279
302004
244075
808002
488007
334034
532035
352002


## Creating and writing a resulting CSV for recommendations for all users in the input database

In [53]:
import pandas as pd
HybridRecommendations=pd.DataFrame([])
t1 = time.time()

total = t1-t0
print(total)

1443.5133612155914


In [54]:
t2 = time.time()
idfam1_to_internal_item_ids_REVERSE = {}
for key, value in idfam1_to_internal_item_ids.items():
    idfam1_to_internal_item_ids_REVERSE[value] = key
k = 10

from numpy import *
u_features = sparse.csr_matrix(full_user_features)
u_rankings = hybrid_model.predict_rank(user_features=u_features,
                                          item_features=item_indicator_features)
    
result = [list(flatnonzero(row  <= k)) for row in u_rankings]
result = numpy.matrix(result)
result = pd.DataFrame(result, columns=[ 'item' + str( i+1) for i in range(k)] )
result['userId'] = idcte_to_internal_user_ids.keys()
result = pd.melt(result, id_vars=['userId'], value_vars=[ 'item' + str( i+1) for i in range(k)])
del result['variable']
result = result.rename(columns={"value": "itemId"})
result['itemId'] = result['itemId'].apply( lambda x: idfam1_to_internal_item_ids_REVERSE[x])
result
table_id = 'Resultados.test_tensorrec_03_porciento_17_junio_2020'
result.to_gbq(table_id, project_id='rmf2gcp')
t3 = time.time()
total = t3-t0
print(total)

1it [00:37, 37.30s/it]

1521.4517896175385


In [55]:
result.head()

,userId,itemId
0,10002,301020
1,10006,874198
2,10011,418073
3,10036,423073
4,10039,541079


### Hybrid Recommender can also be used to predict similar items given some item IDs

In [56]:
hybrid_model.predict_similar_items(item_features=item_indicator_features,item_ids=[3,55,90], n_similar=10) #sera util para los usuarios de compra en efectivo 

[[(3, 0.99999994),
  (383, 0.94644976),
  (810, 0.9398602),
  (838, 0.93781996),
  (965, 0.93415636),
  (2261, 0.9293994),
  (940, 0.92884135),
  (2133, 0.92827964),
  (1428, 0.92538106),
  (1706, 0.92044765)],
 [(55, 0.99999976),
  (1955, 0.98327947),
  (1442, 0.98308074),
  (991, 0.9588507),
  (1119, 0.956786),
  (597, 0.94848186),
  (1790, 0.9456929),
  (1835, 0.9435216),
  (680, 0.9406038),
  (1729, 0.9363351)],
 [(90, 0.9999999),
  (405, 0.97637963),
  (1818, 0.9741595),
  (1107, 0.95932835),
  (1102, 0.95911574),
  (682, 0.9570637),
  (1963, 0.9511348),
  (654, 0.94249254),
  (398, 0.9391309),
  (1473, 0.938779)]]

In [57]:
result.shape

(5695710, 2)